In [77]:
import pandas as pd
import numpy as np

In [96]:
clicks = pd.read_csv("data/clicks.csv.gzip")
# target = pd.read_csv('data/target_final_competencia_revamped.csv', low_memory=False)

In [97]:
# target.loc[:, 'ref_hash'] = target['ref_hash'].map(lambda x: str(x)[:-3])

In [98]:
# clicks['ref_hash'] = clicks['ref_hash'].astype(str)

In [23]:
target.head()

,ref_hash,obj
0,1000169251625791232,0
1,1000169251625791232,0
2,1000395625957344640,0
3,1000395625957344640,0
4,1006670001679961600,0


In [99]:
#Me quedo con las columnas que me importan
clicks = clicks.drop(columns=['agent_device', 'os_minor', 'os_major', 'country_code', 'action_id',\
                'brand', 'touchX', 'touchY', 'specs_brand', 'carrier_id', 'longitude', 'latitude', 'trans_id'], axis=1)

Creando Features

In [84]:
#Muestro las columnas con las que me quedo
clicks.head()

,advertiser_id,source_id,created,wifi_connection,timeToClick,ref_hash,ref_type_android,ref_type_iphone
0,1,2,2019-04-18T05:27:42.197Z,0,2.317,1293710398598742392,1,0
1,1,1,2019-04-18T05:27:03.164Z,1,7.653,1663930990551616564,1,0
2,1,1,2019-04-18T05:42:07.926Z,1,464.796,8488038938665586188,1,0
3,1,1,2019-04-18T05:26:04.446Z,1,225.311,6488361690105189959,1,0
4,1,1,2019-04-18T05:23:37.764Z,1,84.736,1348993302102753419,1,0


In [100]:
clicks['wifi_connection'] = clicks['wifi_connection'].map({True: 1, False: 0})
clicks['wifi_connection'] = clicks['wifi_connection'].fillna(-1)

In [101]:
clicks['created'] = pd.to_datetime(clicks['created'])

In [102]:
clicks['ref_type_android'] = (clicks['ref_type'].astype(str).str.contains("1891515180541284343") == True).astype(int)
clicks['ref_type_iphone'] = (clicks['ref_type'].astype(str).str.contains("1494519392962156891") == True).astype(int)

clicks = clicks.drop('ref_type', axis=1)

In [98]:
clicks['advertiser_id'].value_counts()

2    36135
1    27433
0      516
3      211
4        1
Name: advertiser_id, dtype: int64

In [103]:
clicks['advertiser_0'] = (clicks['advertiser_id'].astype(str).str.contains("0") == True).astype(int)
clicks['advertiser_1'] = (clicks['advertiser_id'].astype(str).str.contains("1") == True).astype(int)
clicks['advertiser_2'] = (clicks['advertiser_id'].astype(str).str.contains("2") == True).astype(int)
clicks['advertiser_3'] = (clicks['advertiser_id'].astype(str).str.contains("3") == True).astype(int)
clicks['advertiser_4'] = (clicks['advertiser_id'].astype(str).str.contains("4") == True).astype(int)

clicks = clicks.drop('advertiser_id', axis=1)

In [100]:
clicks['source_id'].value_counts()

1    33695
0    17155
3     5639
2     3878
6     2186
4      823
5      454
7      329
8      137
Name: source_id, dtype: int64

In [104]:
clicks['source_0'] = (clicks['source_id'].astype(str).str.contains("0") == True).astype(int)
clicks['source_1'] = (clicks['source_id'].astype(str).str.contains("1") == True).astype(int)
clicks['source_2'] = (clicks['source_id'].astype(str).str.contains("2") == True).astype(int)
clicks['source_3'] = (clicks['source_id'].astype(str).str.contains("3") == True).astype(int)
clicks['source_4'] = (clicks['source_id'].astype(str).str.contains("4") == True).astype(int)
clicks['source_5'] = (clicks['source_id'].astype(str).str.contains("5") == True).astype(int)
clicks['source_6'] = (clicks['source_id'].astype(str).str.contains("6") == True).astype(int)
clicks['source_7'] = (clicks['source_id'].astype(str).str.contains("7") == True).astype(int)
clicks['source_8'] = (clicks['source_id'].astype(str).str.contains("8") == True).astype(int)

clicks = clicks.drop('source_id', axis=1)

In [102]:
#clicks['feriado'] = (clicks['created'].dt.day == 19).map({True: 1, False: 0})
#clicks['feriado'] = clicks['feriado'].fillna(-1)

In [105]:
clicks['weekday'] = clicks['created'].dt.weekday

clicks = pd.get_dummies(clicks, columns=['weekday'])

In [106]:
clicks = clicks.rename(index=str, columns={"weekday_0": "lunes", "weekday_1": "martes", "weekday_2": "miercoles", \
                                           "weekday_3": "jueves", "weekday_4": "viernes", "weekday_5": "sabado", \
                                           "weekday_6": "domingo"})

In [107]:
bins = [0, 6, 13, 17, 25]
labels = ['morning','afternoon','evening','night']
hours = clicks['created'].dt.hour
clicks['time'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)

clicks = pd.get_dummies(clicks, columns=['time'])

In [108]:
clicks['feriado'] = (clicks['created'].dt.day).map({19: 1})
clicks['feriado'] = clicks['feriado'].fillna(0)

In [109]:
# clicks = target.merge(clicks, on='ref_hash', how='inner')

In [110]:
clicks.head()

,ref_hash,obj,created,wifi_connection,timeToClick,ref_type_android,ref_type_iphone,advertiser_0,advertiser_1,advertiser_2,...,miercoles,jueves,viernes,sabado,domingo,time_morning,time_afternoon,time_evening,time_night,feriado
0,71410240738361696,0,2019-04-20 17:18:34.033000+00:00,1,5.531,1,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0.0
1,71410240738361696,0,2019-04-22 05:23:06.657000+00:00,1,NaN,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0.0
2,71410240738361696,0,2019-04-20 17:18:34.033000+00:00,1,5.531,1,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0.0
3,71410240738361696,0,2019-04-22 05:23:06.657000+00:00,1,NaN,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0.0


In [112]:
clicks['created'].value_counts().sum()

4

In [74]:
window_1 = clicks.loc[clicks['created'].between('2019-04-18', '2019-04-21')]
window_2 = clicks.loc[clicks['created'].between('2019-04-19', '2019-04-22')]
window_3 = clicks.loc[clicks['created'].between('2019-04-20', '2019-04-23')]
window_4 = clicks.loc[clicks['created'].between('2019-04-21', '2019-04-24')]
window_5 = clicks.loc[clicks['created'].between('2019-04-22', '2019-04-25')]

In [75]:
# Contar la cantidad de apariciones que tiene un usuario por ventada

window_1.loc[:,'count_clicks'] = window_1.groupby('ref_hash')['ref_hash'].transform('count')
window_2.loc[:,'count_clicks'] = window_2.groupby('ref_hash')['ref_hash'].transform('count')
window_3.loc[:,'count_clicks'] = window_3.groupby('ref_hash')['ref_hash'].transform('count')
window_4.loc[:,'count_clicks'] = window_4.groupby('ref_hash')['ref_hash'].transform('count')
window_5.loc[:,'count_clicks'] = window_5.groupby('ref_hash')['ref_hash'].transform('count')

/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [49]:
# primera y ultima instalacion por ref_hash

window_1.loc[:,'last_click'] = window_1.groupby('ref_hash')['created'].transform('max')
window_1.loc[:,'first_click'] = window_1.groupby('ref_hash')['created'].transform('min')

window_2.loc[:,'last_click'] = window_2.groupby('ref_hash')['created'].transform('max')
window_2.loc[:,'first_click'] = window_2.groupby('ref_hash')['created'].transform('min')

window_3.loc[:,'last_click'] = window_3.groupby('ref_hash')['created'].transform('max')
window_3.loc[:,'first_click'] = window_3.groupby('ref_hash')['created'].transform('min')

window_4.loc[:,'last_click'] = window_4.groupby('ref_hash')['created'].transform('max')
window_4.loc[:,'first_click'] = window_4.groupby('ref_hash')['created'].transform('min')

window_5.loc[:,'last_click'] = window_5.groupby('ref_hash')['created'].transform('max')
window_5.loc[:,'first_click'] = window_5.groupby('ref_hash')['created'].transform('min')

/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/pandas/core/indexing.py:362: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  self.obj[key] = _infer_fill_value(value)


In [50]:
# diferencia de tiempo entre el primero y el ultimo click

window_1.loc[:,'diff_between_clicks'] = (window_1['last_click'] - window_1['first_click']).dt.total_seconds()

window_2.loc[:,'diff_between_clicks'] = (window_2['last_click'] - window_2['first_click']).dt.total_seconds()

window_3.loc[:,'diff_between_clicks'] = (window_3['last_click'] - window_3['first_click']).dt.total_seconds()

window_4.loc[:,'diff_between_clicks'] = (window_4['last_click'] - window_4['first_click']).dt.total_seconds()

window_5.loc[:,'diff_between_clicks'] = (window_5['last_click'] - window_5['first_click']).dt.total_seconds()

In [51]:
# Promedio de diferencias de tiempo entre todos los clicks
window_1.loc[:,'prom_time_click'] = 0
window_1.loc[window_1['count_clicks'] > 1, 'prom_time_click'] = ((window_1['last_click'] - window_1['first_click'])/ \
                                 (window_1['count_clicks'] -1)).dt.total_seconds()

window_2.loc[:,'prom_time_click'] = 0
window_2.loc[window_2['count_clicks'] > 1,'prom_time_click'] = ((window_2['last_click'] - window_2['first_click'])/ \
                                 (window_2['count_clicks'] - 1)).dt.total_seconds()

window_3.loc[:,'prom_time_click'] = 0
window_3.loc[window_3['count_clicks'] > 1,'prom_time_click'] = ((window_3['last_click'] - window_3['first_click'])/ \
                                 (window_3['count_clicks'] - 1)).dt.total_seconds()

window_4.loc[:,'prom_time_click'] = 0
window_4.loc[window_4['count_clicks'] > 1, 'prom_time_click'] = ((window_4['last_click'] - window_4['first_click'])/ \
                                 (window_4['count_clicks'] - 1)).dt.total_seconds()

window_5.loc[:,'prom_time_click'] = 0
window_5.loc[window_5['count_clicks'] > 1, 'prom_time_click'] = ((window_5['last_click'] - window_5['first_click'])/ \
                                 (window_5['count_clicks'] -1 )).dt.total_seconds()

In [52]:

window_1 = window_1.sort_values('created')
window_2 = window_2.sort_values('created')
window_3 = window_3.sort_values('created')
window_4 = window_4.sort_values('created')
window_5 = window_5.sort_values('created')

In [53]:
window_1 = window_1.groupby('ref_hash').head(1)
window_2 = window_2.groupby('ref_hash').head(1)
window_3 = window_3.groupby('ref_hash').head(1)
window_4 = window_4.groupby('ref_hash').head(1)
window_5 = window_5.groupby('ref_hash').head(1)

In [54]:
print(len(window_1))
print(len(window_2))
print(len(window_3))
print(len(window_4))
print(len(window_5))

1
1
1
1
1


In [32]:
window_1 = window_1.drop(columns=['created', 'last_click', 'first_click'], axis=1)
window_2 = window_2.drop(columns=['created', 'last_click', 'first_click'], axis=1)
window_3 = window_3.drop(columns=['created', 'last_click', 'first_click'], axis=1)
window_4 = window_4.drop(columns=['created', 'last_click', 'first_click'], axis=1)
window_5 = window_5.drop(columns=['created', 'last_click', 'first_click'], axis=1)

KeyError: "['created' 'last_click' 'first_click'] not found in axis"

In [34]:
window_1 = window_1.drop(columns=['ref_type'], axis=1)
window_2 = window_2.drop(columns=['ref_type'], axis=1)
window_3 = window_3.drop(columns=['ref_type'], axis=1)
window_4 = window_4.drop(columns=['ref_type'], axis=1)
window_5 = window_5.drop(columns=['ref_type'], axis=1)

In [35]:
window_1.head()

,advertiser_id,wifi_connection,timeToClick,ref_hash,source_0,source_1,source_2,source_3,source_4,source_5,...,sabado,domingo,time_morning,time_afternoon,time_evening,time_night,feriado,count_clicks,diff_between_clicks,prom_time_click
23353,2.0,1.0,297.701,4.641600e+18,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,245066.334,245066.334
20507,2.0,1.0,8.169,3.242558e+18,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,44634.388,44634.388
49715,1.0,1.0,88.512,2.019857e+18,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,49220.151,49220.151
50228,1.0,1.0,133.935,4.330321e+18,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000,0.000
25341,2.0,0.0,547.220,4.697413e+18,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000,0.000


In [36]:
window_1.to_csv(path_or_buf='windows_click/window_1.csv',index=False)
window_2.to_csv(path_or_buf='windows_click/window_2.csv',index=False)
window_3.to_csv(path_or_buf='windows_click/window_3.csv',index=False)
window_4.to_csv(path_or_buf='windows_click/window_4.csv',index=False)
window_5.to_csv(path_or_buf='windows_click/window_5.csv',index=False)